## Import Libraries and Modules

In [4]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 

from sklearn.metrics import accuracy_score

#import mediapipe
import mediapipe as mp

#import OpenCV
import cv2

In [5]:
#utility that draws the whole connection
mp_drawings = mp.solutions.drawing_utils

#mediapipe solutions.landmarks from holistic
mp_holistic = mp.solutions.holistic

### Contents:
- [Background](#Background)
- [Data Collection & Cleaning](#Data-Collection-&-Cleaning)
- [Preprocessing and Feature Engineering](#Preprocessing-and-Feature-Engineering)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)


## Background

## Data Collection & Cleaning

### Make Detections

In [30]:
#Video Feed

# Setting up video capture device
# 0 = built-in webcam & 2 = External webcam
# If want to grab a video clip, type the name of video and its file loc within (...)
cap = cv2.VideoCapture(0)

# Start the holistic model
# 'Holistic' allows me to access the Holistic estimation model
# '0.6' = 60%
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    
    #While loop to allow video to stay open till I give command to close window
    while cap.isOpened():
        #frame is the frame that will be returning
        #cap.read is where the webcam will capture and read the video
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #apply and set the image to be unwriteable
        image.flags.writeable = False
        

        #Make Detections
        results = holistic.process(image)

#         #To show the coords of each types of landmarks (face, pose, hand), add '.face_landmarks' at end of "results" variable
#         print(results)

        #setting image writeable back to true to be able process it
        image.flags.writeable = True

        #recoloring it back to BGR b/c it will rerender back to opencv
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         #Draw the face landmarks
#         mp_drawings.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
#                                   mp_drawings.DrawingSpec(color=(0,155,0), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(100,155,0), thickness=1, circle_radius=1))
        
        
#         #Draw the right hand landmarks
#         mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawings.DrawingSpec(color=(0,155,155), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(100,155,0), thickness=2, circle_radius=1))
        
#         #Draw the left hand landmarks
#         mp_drawings.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawings.DrawingSpec(color=(200,155,50), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(0,255,88), thickness=2, circle_radius=1))
        
        #Draw the pose landmarks
        mp_drawings.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawings.DrawingSpec(color=(255,155,255), thickness=1, circle_radius=1),
                                  mp_drawings.DrawingSpec(color=(100,155,200), thickness=2, circle_radius=1))

        #separate window will pop open to show the video feed
        #'Live Feed' is the name of the window
        cv2.imshow('Raw Live Feed', image)

        #how to turn it off
        #the 'e' letter will exit the live feed window
        if cv2.waitKey(10) & 0xFF == ord('e'):
            break
        
#release the camera feed
cap.release()

#close the window
cv2.destroyAllWindows()

In [7]:
results.pose_landmarks.landmark

[x: 0.49802494049072266
y: 0.12633447349071503
z: -3.044553279876709
visibility: 0.9558461904525757
, x: 0.5660862326622009
y: -0.001154470955953002
z: -3.0741236209869385
visibility: 0.9146250486373901
, x: 0.6021206378936768
y: -0.009166738018393517
z: -3.074179172515869
visibility: 0.9032266139984131
, x: 0.6346606016159058
y: -0.017070991918444633
z: -3.0746963024139404
visibility: 0.8879812955856323
, x: 0.4414237141609192
y: -0.016044240444898605
z: -3.0711495876312256
visibility: 0.9470598697662354
, x: 0.403998464345932
y: -0.036127496510744095
z: -3.0715670585632324
visibility: 0.9526250958442688
, x: 0.370451420545578
y: -0.05634044110774994
z: -3.0719895362854004
visibility: 0.9604048728942871
, x: 0.6971777081489563
y: -0.029930435121059418
z: -2.674870014190674
visibility: 0.8597866892814636
, x: 0.3054342269897461
y: -0.09471011161804199
z: -2.639482259750366
visibility: 0.971000611782074
, x: 0.5523075461387634
y: 0.1951964944601059
z: -2.8443126678466797
visibility: 0.9

![Image](test.jpg)

### Export coordinates to csv

In [8]:
import csv
import os

In [9]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [10]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
    
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [11]:
landmarks[-1]

'v33'

In [12]:
# with open('coords.csv', mode='w', newline='') as f:
#     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     csv_writer.writerow(landmarks)

In [26]:
class_name = 'Rest'

In [29]:
import time

# Delay for 5 seconds
time.sleep(5)

#Video Feed

# Setting up video capture device
# 0 = built-in webcam & 2 = External webcam
# If want to grab a video clip, type the name of video and its file loc within (...)
cap = cv2.VideoCapture(0)

# Start the holistic model
# 'Holistic' allows me to access the Holistic estimation model
# '0.6' = 60%
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    
    #While loop to allow video to stay open till I give command to close window
    while cap.isOpened():
        #frame is the frame that will be returning
        #cap.read is where the webcam will capture and read the video
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #apply and set the image to be unwriteable
        image.flags.writeable = False
        

        #Make Detections
        results = holistic.process(image)

#         #To show the coords of each types of landmarks (face, pose, hand), add '.face_landmarks' at end of "results" variable
#         print(results)

        #setting image writeable back to true to be able process it
        image.flags.writeable = True

        #recoloring it back to BGR b/c it will rerender back to opencv
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         #Draw the face landmarks
#         mp_drawings.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
#                                   mp_drawings.DrawingSpec(color=(0,155,0), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(100,155,0), thickness=1, circle_radius=1))
        
        
#         #Draw the right hand landmarks
#         mp_drawings.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawings.DrawingSpec(color=(0,155,155), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(100,155,0), thickness=2, circle_radius=1))
        
#         #Draw the left hand landmarks
#         mp_drawings.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawings.DrawingSpec(color=(200,155,50), thickness=1, circle_radius=1),
#                                   mp_drawings.DrawingSpec(color=(0,255,88), thickness=2, circle_radius=1))
        
        #Draw the pose landmarks
        mp_drawings.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawings.DrawingSpec(color=(255,155,255), thickness=1, circle_radius=1),
                                  mp_drawings.DrawingSpec(color=(100,155,200), thickness=2, circle_radius=1))
        
        
        #Export coordinates to csv
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            row = pose_row
            row.insert(0, class_name)
            
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
                
        except:
            pass

        #separate window will pop open to show the video feed
        #'Live Feed' is the name of the window
        cv2.imshow('Raw Live Feed', image)

        #how to turn it off
        #the 'e' letter will exit the live feed window
        if cv2.waitKey(10) & 0xFF == ord('e'):
            break
        
#release the camera feed
cap.release()

#close the window
cv2.destroyAllWindows()

In [1]:
# pose = results.pose_landmarks.landmark

In [42]:
np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()

array([ 0.95066994, -0.01461398, -0.09395941,  0.98934788,  0.94883269,
       -0.03829821, -0.08437638,  0.97932404,  0.95030415, -0.03859024,
       -0.08471672,  0.97477216,  0.95184308, -0.03873914, -0.08474321,
        0.97516853,  0.94153321, -0.03932981, -0.12907505,  0.98826635,
        0.93696892, -0.04054881, -0.12914185,  0.98911673,  0.93240994,
       -0.04179414, -0.13005906,  0.99164701,  0.93863189, -0.03858279,
        0.05792949,  0.96118206,  0.91256225, -0.04089028, -0.1395576 ,
        0.99551028,  0.94703645,  0.00483351, -0.02820437,  0.98760986,
        0.93749684,  0.00364319, -0.08611027,  0.99509621,  0.93373585,
        0.07686644,  0.23508433,  0.99576211,  0.85452271,  0.02333002,
       -0.23811035,  0.9997564 ,  0.91466981,  0.20957667,  0.32541013,
        0.19608799,  0.75160372,  0.13816597, -0.30939317,  0.98667759,
        0.93541336,  0.27068728,  0.18977471,  0.24317396,  0.72868669,
        0.3275027 , -0.36414224,  0.96897143,  0.94616205,  0.28

In [1]:
# pose = results.pose_landmarks.landmark
# pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

# row = pose_row
# row.insert(0, class_name)

In [44]:
len(row)

133

In [45]:
1 + 4*33

133

## Verify the data collected

In [46]:
df = pd.read_csv("coords.csv")
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Rest,0.429787,0.124724,-0.288868,0.999871,0.439223,0.105139,-0.294509,0.999860,0.444510,...,0.345616,0.529908,0.421035,0.920161,-0.068212,0.890566,0.425664,0.896875,0.259089,0.557966
1,Rest,0.429987,0.124991,-0.259555,0.999874,0.439233,0.105773,-0.269675,0.999863,0.444493,...,0.337917,0.527592,0.420221,0.921053,-0.073450,0.891321,0.425892,0.897638,0.257494,0.556362
2,Rest,0.430823,0.125169,-0.263092,0.999880,0.439768,0.106144,-0.272165,0.999869,0.444812,...,0.333816,0.528580,0.419921,0.921928,-0.090104,0.894069,0.425914,0.898530,0.255306,0.560090
3,Rest,0.431291,0.125154,-0.259533,0.999884,0.440098,0.106258,-0.268748,0.999874,0.445015,...,0.334046,0.524296,0.419030,0.922586,-0.087694,0.893417,0.426566,0.898410,0.255965,0.554980
4,Rest,0.431531,0.125360,-0.263824,0.999889,0.440243,0.106612,-0.272191,0.999879,0.445120,...,0.321103,0.526335,0.418745,0.923012,-0.108563,0.896314,0.427117,0.898055,0.242768,0.560145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,Down,0.515438,0.377535,-0.053847,0.999826,0.507816,0.363264,-0.038964,0.999799,0.507193,...,-0.100868,0.979067,0.499722,0.844744,0.381113,0.674132,0.501714,0.891181,-0.154394,0.977601
851,Down,0.515385,0.377076,-0.047643,0.999837,0.507336,0.363130,-0.033030,0.999809,0.506635,...,-0.098392,0.979381,0.498543,0.843097,0.365697,0.671934,0.501400,0.890624,-0.143058,0.977627
852,Down,0.514902,0.377082,-0.021066,0.999850,0.507074,0.363163,-0.007410,0.999823,0.506363,...,-0.073316,0.979416,0.498285,0.843293,0.329243,0.674347,0.501519,0.889914,-0.086834,0.977888
853,Down,0.515605,0.377088,-0.035693,0.999835,0.507524,0.363147,-0.020351,0.999810,0.506713,...,-0.020297,0.976795,0.498319,0.843766,0.306504,0.665239,0.501611,0.889117,-0.019261,0.975555


In [47]:
df['class'].value_counts()

Down          307
Rest          306
Transition    242
Name: class, dtype: int64